## Include libraries

In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sksurv.column import encode_categorical

## dataset loading...

In [2]:
surv_data = pd.read_csv(os.path.join('..', '..', 'data', '(v4)STROKE_VITAL_SIGN_missForest.csv'))

In [3]:
surv_data = surv_data[surv_data['CVDeath'] != 9999]

## data preprocessing

In [4]:
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper

def data_processing(data_df):
    data_df_x = data_df.drop(['LOC', 'UID', 'Mortality', 'SurvivalDays', 'CVDeath', 'SurvivalMonths',
                              'admission_date', 'discharge_date', 'death_date'], axis=1)

    data_df_y = data_df[['CVDeath', 'SurvivalDays']]

    X_temp = data_df_x[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    y_temp = data_df_y[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    X_df_train, X_df_val, y_df_train, y_df_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=369)

    X_df_test_kao = data_df_x[data_df.LOC == '8']
    y_df_test_kao = data_df_y[data_df.LOC == '8']

    categorical_columns = ['Sex', 'AF', 'DM', 'HTN', 'Dyslipidemia', 'CHF', 'Smoking',
                           'Cancer.before.adm']
    
    numerical_columns = np.setdiff1d(data_df_x.columns, categorical_columns).tolist()

    categorical_ix = [data_df_x.columns.get_loc(col) for col in categorical_columns]
    numerical_ix =  np.setdiff1d(list(range(0, len(data_df_x.columns))), categorical_ix).tolist()

    scaler = preprocessing.StandardScaler()

    standardize = [([col], scaler) for col in numerical_columns]
    leave = [(col, None) for col in categorical_columns]

    x_mapper = DataFrameMapper(standardize + leave)

    X_df_train = pd.DataFrame(data=x_mapper.fit_transform(X_df_train),
                              columns=numerical_columns+categorical_columns,
                              index=X_df_train.index)

    X_df_val = pd.DataFrame(data=x_mapper.fit_transform(X_df_val),
                            columns=numerical_columns+categorical_columns,
                            index=X_df_val.index)

    X_df_test_kao = pd.DataFrame(data=x_mapper.fit_transform(X_df_test_kao),
                                 columns=numerical_columns+categorical_columns,
                                 index=X_df_test_kao.index)
    # since those categorical variable are all boolen... no need to one-hot them
    # https://stackoverflow.com/questions/43515877/should-binary-features-be-one-hot-encoded
    
#     X_df_train = encode_categorical(X_df_train, columns=categorical_columns)
#     X_df_val = encode_categorical(X_df_val, columns=categorical_columns)
#     X_df_test_kao = encode_categorical(X_df_test_kao, columns=categorical_columns)
    
    return X_df_train, X_df_val, y_df_train, y_df_val, X_df_test_kao, y_df_test_kao


get_target = lambda df: (df['SurvivalDays'].values, df['CVDeath'].values)

In [5]:
X_train, X_val, y_train, y_val, X_test_kao, y_test_kao = data_processing(surv_data)

In [6]:
print(X_train)

            ALT       Age      CHOL       CKD   DBPCV.G    HRCV.G  Mean.PP.G  \
13943  0.361083  0.222219  1.115646 -1.210555 -0.305294  2.102906  -0.770882   
7658  -0.058095  1.139428 -1.036461  0.079579 -0.305294  2.102906   0.555585   
15136 -0.351520 -0.542121  0.513982  0.079579 -0.305294 -1.040582  -0.770882   
902    0.109576 -2.147237  0.351995 -1.210555 -0.305294  0.007248  -0.107648   
12929  0.319165 -0.007083 -0.388515 -1.210555 -0.305294  0.007248  -0.770882   
...         ...       ...       ...       ...       ...       ...        ...   
10870  0.612590 -0.924292  2.156989 -1.210555 -0.305294 -1.040582  -0.770882   
6657   1.115604 -0.924292 -0.735629 -1.210555 -0.305294 -1.040582   0.555585   
2130   0.822179  0.527956 -1.337294  1.369714  0.817380  1.055077  -0.770882   
397   -0.351520 -1.306462  0.166868 -1.210555 -0.305294 -1.040582  -1.434116   
10878  3.924098 -0.542121  0.537123  0.079579 -0.305294  1.055077   0.555585   

       Mean.SBP.G  MeanDBP.G  MeanHR.G 

## Feature selection

In [7]:
rf_list = pd.read_csv(os.path.join('rfs_FS_cvd.csv')).sort_values(by='weight', ascending=False).feature


## Deepsur

In [8]:
import torch
import torchtuples as tt
from pycox.evaluation import EvalSurv
from pycox.models import CoxPH

def Deepsur(Xtrain, Xval, Ytrain, Yval):    
    # preprocessing data
    Xtrain = Xtrain.values.astype('float32')
    Xval = Xval.values.astype('float32')
    Ytrain = get_target(Ytrain)
    Yval = get_target(Yval)
    val = Xval, Yval
    
    # parameters
    in_features = Xtrain.shape[1]
    num_nodes = [25, 25]
    out_features = 1
    batch_norm = True
    dropout = 0.1
    output_bias = False
    batch_size = 128
    if Xtrain.shape[0]%batch_size == 1:
        batch_size = batch_size - 1
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=20)]
    verbose = False
    
    # network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)
    model = CoxPH(net, tt.optim.Adam)
    
    # train
    lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
    #best_lr = lrfinder.get_best_lr()
    #_ = lrfinder.plot()
    model.optimizer.set_lr(0.01)
    log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
                    val_data=val, val_batch_size=batch_size)
    return model
    

### DeepSur

In [9]:
durations_test_kao, events_test_kao = get_target(y_test_kao)

for i in range(len(rf_list)):
    seed = 369
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    #
    features = rf_list.iloc[0:i+1].values
    all_features = X_train.columns.values
    drop_features = np.setdiff1d(all_features, features)
    if len(features) > 0:   
        X_train_fs = X_train.drop(drop_features, axis=1)
        X_val_fs = X_val.drop(drop_features, axis=1)
        X_test_kao_fs = X_test_kao.drop(drop_features, axis=1).values.astype('float32')
        
        deep_sur = Deepsur(X_train_fs, X_val_fs, y_train, y_val)
        # prediction
        _ = deep_sur.compute_baseline_hazards()
        surv_kao = deep_sur.predict_surv_df(X_test_kao_fs)
        # evaluation
        ev_kao = EvalSurv(surv_kao, durations_test_kao, events_test_kao, censor_surv='km')
        print(X_train_fs.shape[1])
        print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

1
Kao C-index = 0.728
2
Kao C-index = 0.792
3
Kao C-index = 0.806
4
Kao C-index = 0.815
5
Kao C-index = 0.819
6
Kao C-index = 0.818
7
Kao C-index = 0.823
8
Kao C-index = 0.825
9
Kao C-index = 0.822
10
Kao C-index = 0.826
11
Kao C-index = 0.823
12
Kao C-index = 0.822
13
Kao C-index = 0.824
14
Kao C-index = 0.823
15
Kao C-index = 0.825
16
Kao C-index = 0.822
17
Kao C-index = 0.823
18
Kao C-index = 0.822
19
Kao C-index = 0.821
20
Kao C-index = 0.825
21
Kao C-index = 0.820
22
Kao C-index = 0.828
23
Kao C-index = 0.826
24
Kao C-index = 0.822
25
Kao C-index = 0.825


## RandomSurvivalForest

In [10]:
from sksurv.ensemble import RandomSurvivalForest

for i in range(len(rf_list)):
    features = rf_list.iloc[0:i+1].values
    if len(features) > 0:
        X_train_fs = X_train[features]
        X_val_fs = X_val[features]
        X_test_kao_fs = X_test_kao[features].values.astype('float32')
        
        y_train_cox =  y_train.copy()
        y_train_cox.loc[:, 'CVDeath'] = y_train_cox['CVDeath'].astype(bool)
        y_train_cox = np.array(list(y_train_cox.to_records(index=False)))
        y_test_kao_cox =  y_test_kao.copy()
        y_test_kao_cox.loc[:, 'CVDeath'] = y_test_kao_cox['CVDeath'].astype(bool)
        y_test_kao_cox = np.array(list(y_test_kao_cox.to_records(index=False)))
        
        rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)
        rsf.fit(X_train_fs, y_train_cox)
        print(X_train_fs.shape[1])
        print('Kao C-index = %.3f' %(rsf.score(X_test_kao_fs, y_test_kao_cox)))

1
Kao C-index = 0.728
2
Kao C-index = 0.785
3
Kao C-index = 0.800
4
Kao C-index = 0.804
5
Kao C-index = 0.811
6
Kao C-index = 0.814
7
Kao C-index = 0.818
8
Kao C-index = 0.824
9
Kao C-index = 0.819
10
Kao C-index = 0.820
11
Kao C-index = 0.820
12
Kao C-index = 0.821
13
Kao C-index = 0.821
14
Kao C-index = 0.822
15
Kao C-index = 0.823
16
Kao C-index = 0.823
17
Kao C-index = 0.823
18
Kao C-index = 0.823
19
Kao C-index = 0.822
20
Kao C-index = 0.824
21
Kao C-index = 0.823
22
Kao C-index = 0.823
23
Kao C-index = 0.824
24
Kao C-index = 0.823
25
Kao C-index = 0.822
